# Program 4: Benchmarking Heaps
### Mikhail Filippov

---
In this assigment, we will implement **three** approaches to heapifying an array. Specifically, we are trying to build a max heap. The three approaches will be:
1. Insertion Method: Insert elements one at a time into the heap
2. Bubble down until value is bigger than both children (we'll call this "Bubble Method 1")
3. Bubble down all the way, then bubble up until value is smaller than parent (we'll call this "Bubble Method 2")

With each one, we will benchmark on **sorted**, **reverse sorted**, and **random** arrays to test the `heapify` method and analyze using regression techniques to estimate asymptotic time complexity for each case.

---

### Planned Approach
For our three cases to benchmark, we will be using **sorted**, **reverse sorted**, and **randomly shuffled** lists to heapify with each approach. Generating each list won't be too much of a problem, and we will increase the size of each list linearly to look at the graphs and slopes generated by the dependant time output by the benchmarks.

Expected results and predictions:
1. Insertion method: We know that an insertion takes $O(log(n))$ time, so to insert $n$ number of elements, we should get a time complexity of $O(n\times log(n))$. Since we are inserting at the end of the heap and bubbling up, the expected best case should be a **reverse sorted** list, as we will be adding the biggest element at `A[0]` first, followed by all elements at `A[i]` with $i \gt 0$, with each element `A[i+1]` $\lt$ `A[i]`. This allows for all smaller elements to be added to the bottom and not needed to be bubbled up. This, should, reduce our best case runtime to $O(n)$, as we are adding elements in costant time $n$ number of times now.
2. Bubble Method 1: We know the recurrence relation for this method is $T(n)=2T(\frac{n}{2}) + \Theta(log(n))$. In the best case though, **reverse sorted** again, the $\Theta(log(n))$ part is insignificant. This reduces our recurrence to $T(n)=2T(\frac{n}{2}) + \Theta(1)$ since we still need to call `BubbleDown`. This is still reduced to $O(n)$, but it should theoretically run faster than any other case.
3. Bubble Method 2: This approach's best case should be the **sorted** list. This happens since no matter what, we bubble each value down **all** the way, and bubble back up *until the value is smaller than parent*. In a sorted list, this ensures that all of the smaller values will be bubbled down to the end and won't have to be bubble back up. The asymptotic running time for this should be $O(n)$.

---

### Heapification Approaches
#### 1. Insertion
```py
def insertion_heapify(A):
    """
    Heapify using insertion method and adding elements one by one
    """
    heap = []
    for i in range(len(A)):
        insert(heap, A[i])
    A[:] = heap
```

#### 2. Heapification Function
```py
def heapify(A, bubble_down=bubble_method_1):
    """
    Heapify using the passed in bubble_down method and build the heap buttom up
    """
    heapify_h(A, 0, bubble_down)

def heapify_h(A, i, bubble_down):
    """
    Heapify wrapper calling helper
    """
    if i < len(A)//2:
        heapify_h(A, left(i), bubble_down)
        heapify_h(A, right(i), bubble_down)
        bubble_down(A, i)
```

#### 2a. Bubble Method 1
```py
def bubble_method_1(A, i):
    """
    Bubble down until value is bigger than both children (we'll call this "Bubble Method 1")
    Choose the bigger child and swap with it while comparing with itself, then repeat until no more swaps are needed
    """
    n = len(A)
    l = left(i)
    r = right(i)
    while (l < n and A[i] < A[l]) or (r < n and A[i] < A[r]):
        if l < n and r < n:
            # left
            if A[l] > A[r]:
                temp = A[l]
                A[l] = A[i]
                A[i] = temp
                i = l
            # right
            else:
                temp = A[r]
                A[r] = A[i]
                A[i] = temp
                i = r
        elif l < n:
            # left
            temp = A[l]
            A[l] = A[i]
            A[i] = temp
            i = l
        elif r < n:
            # right
            temp = A[r]
            A[r] = A[i]
            A[i] = temp
            i = r
        l = left(i)
        r = right(i)
```

#### 2b. Bubble Method 2
```py
def bubble_method_2(A, i):
    """
    Bubble down all the way, then bubble up until value is smaller than parent
    Choose the bigger child and swap with it all the way down, then go back up until smaller than parent
    """
    n = len(A)
    l = left(i)
    r = right(i)
    while l < n:
        if r < n:
            # left
            if A[l] > A[r]:
                temp = A[l]
                A[l] = A[i]
                A[i] = temp
                i = l
            # right
            else:
                temp = A[r]
                A[r] = A[i]
                A[i] = temp
                i = r
        else:
            # left
            temp = A[l]
            A[l] = A[i]
            A[i] = temp
            i = l
        l = left(i)
        r = right(i)
    
    # bubble up
    parent_i = parent(i)
    while i > 0 and A[parent_i] < A[i]:
        temp = A[parent_i]
        A[parent_i] = A[i]
        A[i] = temp
        i = parent_i
        parent_i = parent(i)
```

*Note: The appendix will contain the full source code implementations for the three approaches and their respective descriptions, as well as any helper methods not described above.*

---

### Benchmarking

In [7]:
import dill
df = None
with open('heapify_benchmark.db', 'rb') as f:
    df = dill.load(f)